In [2]:
import torch
import os

/home/jj/anaconda3/envs/30env/envs/torch_1.12_2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Difference of global norm

In [6]:
new_global = torch.cat([torch.load(f'/mnt/ssd/processed/S1200/img/996782/frame_{i}.pt') for i in range(20)], dim=3)
prev_global = torch.cat([torch.load(f'/home/jj/996782/global_normalize/rfMRI_LR_TR_{i}.pt') for i in range(20)], dim=3)

idx = (prev_global.flatten() - new_global.flatten()).argmax()

print('maximum difference value:', prev_global.flatten()[idx] - new_global.flatten()[idx])
print('avg absolute minimum:', (prev_global - new_global).abs().mean())

maximum difference value: tensor(0.0020)
avg absolute minimum: tensor(0.0004)


# Difference of Voxel norm

In [10]:
fmri.shape

torch.Size([75, 93, 81, 20])

In [33]:
mean = torch.load(f'/mnt/ssd/processed/S1200/img/996782/voxel_mean.pt')
std = torch.load(f'/mnt/ssd/processed/S1200/img/996782/voxel_std.pt')
fmri = torch.cat([torch.load(f'/mnt/ssd/processed/S1200/img/996782/frame_{i}.pt') for i in range(20)], dim=3)

new_voxel = torch.zeros_like(fmri)

background_value = fmri.flatten()[0]
background = fmri == background_value

new_voxel[~background] = ((fmri - mean) / std)[~background]
new_voxel.add_(background * new_voxel.min()) # inplace operation

prev_voxel = torch.cat([torch.load(f'/home/jj/996782/per_voxel_normalize/rfMRI_LR_TR_{i}.pt') for i in range(20)], dim=3)
prev_voxel[prev_voxel!=prev_voxel] = prev_voxel[(prev_voxel==prev_voxel)].min()

idx = (prev_voxel.flatten() - new_voxel.flatten()).argmax()

print('maximum difference value:', prev_voxel.flatten()[idx] - new_voxel.flatten()[idx])
print('avg absolute minimum:', (prev_voxel - new_voxel).abs().mean())

diff = prev_voxel - new_voxel
print('diff around boundary:', len(diff[diff.abs()>0.05]))
print('diff_smaller_than_6', diff[diff.abs()<4].abs().mean())

maximum difference value: tensor(4.8805)
avg absolute minimum: tensor(0.0063)
diff around boundary: 625
diff_smaller_than_6 tensor(0.0063)
